In [1]:
import torch
import torch.nn as nn

In [2]:
a = torch.randn(3,3)
drop = nn.Dropout(p=0.5)
a

tensor([[ 0.2728,  0.0327,  0.0908],
        [-0.2410,  0.1046,  0.5711],
        [-0.4799,  1.8335,  0.6909]])

In [3]:
drop(a)

tensor([[0.0000, 0.0000, 0.1817],
        [-0.0000, 0.0000, 0.0000],
        [-0.0000, 0.0000, 1.3817]])

In [4]:
bsz = 3
seqlen= 4
num_experts = 3
dim = 1024
rank=16


In [5]:
x = torch.randn(bsz,seqlen,dim)

In [6]:
linearA = nn.Linear(dim,rank,bias=False)
linearB = nn.Linear(rank,dim,bias=False)
core1 = nn.Parameter(torch.randn(rank,rank))
core2 = nn.Parameter(torch.randn(rank,rank))
core3 = nn.Parameter(torch.randn(rank,rank))
router = nn.Linear(dim,num_experts)

In [7]:
core1

Parameter containing:
tensor([[ 0.1862,  0.6763,  1.2091, -1.2465,  0.4353,  1.4144, -0.5171, -0.6876,
          1.2075, -0.5523, -0.8428,  0.1183,  0.2166, -0.8014,  1.6328, -0.1063],
        [ 0.3325, -0.7579, -0.9133, -0.4080,  0.5192,  0.3948, -0.3663,  1.3122,
          0.1303, -0.0418, -1.8189,  0.4248, -0.1723, -0.5713, -0.5094,  0.8410],
        [-1.3499, -1.5122, -0.5074, -0.7852, -1.2961, -0.5590,  0.9056,  0.0041,
          1.7375, -1.2687,  1.0841,  0.4440, -1.7731,  0.1584, -1.5822,  0.4035],
        [-1.7012,  2.6138, -0.7424, -1.2565,  1.7206,  1.7479,  3.0169,  0.0604,
          0.6764,  0.0190,  1.9124,  0.9740, -0.1615,  1.6708, -0.4097,  0.2884],
        [ 1.2708,  1.1654,  0.4071,  0.2341, -0.1337, -0.6656,  1.2202,  0.9437,
          0.5608, -0.5309,  0.4141, -0.5244,  0.2199, -0.6871, -0.8821,  0.1413],
        [-1.1298,  0.2032, -0.0188,  1.1780,  0.4541,  2.0186, -1.3114,  1.9769,
          0.3003, -0.1126,  0.8949, -1.3592, -0.2902, -1.1332, -0.0161, -0.3478],


In [8]:
router_weight = router(x)

In [9]:
router_weight.shape

torch.Size([3, 4, 3])

In [10]:
t = linearA(x)
h = linearB(t @ core1) * router_weight[:,:,0].unsqueeze(-1) + linearB(t @ core2) * router_weight[:,:,1].unsqueeze(-1) + linearB(t @ core3) * router_weight[:,:,2].unsqueeze(-1)

In [11]:
h

tensor([[[ 0.1364,  0.3528, -0.4884,  ...,  0.0451, -0.1590,  0.6033],
         [ 0.9869, -1.3004, -1.6613,  ...,  0.3666, -1.1871,  0.4440],
         [-0.6000,  0.4119,  0.8842,  ...,  0.1745,  0.2319,  0.6379],
         [-0.0575,  0.8677,  2.0496,  ..., -1.1423,  0.7695, -2.2850]],

        [[ 1.2993, -0.9309, -1.9751,  ..., -0.8612, -0.3816, -4.5359],
         [-0.1006,  0.9073, -1.0636,  ...,  0.1978, -0.8493, -0.4151],
         [-0.4874,  0.2984,  0.0530,  ...,  0.2068, -0.2109,  0.3485],
         [ 0.1081, -0.6814,  0.3874,  ..., -0.1913, -0.1430, -0.3207]],

        [[-0.7838,  0.8841, -0.1883,  ...,  0.5851,  1.0157, -0.8078],
         [-0.8792,  1.5166,  0.8756,  ..., -0.7423, -0.0695, -0.1658],
         [-0.1608, -0.1352, -0.1841,  ..., -0.5653, -0.5929, -0.6947],
         [-3.1612,  1.2398,  1.7035,  ..., -1.5426, -0.4911, -1.5156]]],
       grad_fn=<AddBackward0>)

In [15]:
test = core1 * router_weight[:,:,0].unsqueeze(-1).unsqueeze(-1)

In [17]:
test.shape

torch.Size([3, 4, 16, 16])

In [18]:
core = (core1 * router_weight[:,:,0].unsqueeze(-1).unsqueeze(-1) + core2 * router_weight[:,:,1].unsqueeze(-1).unsqueeze(-1) + core3 * router_weight[:,:,2].unsqueeze(-1).unsqueeze(-1))

In [23]:
core.shape

torch.Size([3, 4, 16, 16])

In [39]:
h2_=torch.einsum('bsrr,bsr->bsr',core,linearA(x) )

In [40]:
h3_ = linearB(h2_)

In [41]:
h3_

tensor([[[-0.1329,  0.2742, -0.3628,  ..., -0.2147, -0.1812,  0.4554],
         [ 0.2041,  0.0353, -0.5267,  ..., -0.1674,  0.2409,  0.8286],
         [-0.0753,  0.0321,  0.1317,  ..., -0.1282,  0.0046,  0.0396],
         [ 0.9561, -0.0449,  0.0016,  ..., -0.0255,  0.3517,  0.0215]],

        [[ 0.5108, -0.6670, -0.1952,  ..., -0.0209, -0.1151,  0.2315],
         [-0.3230,  0.2307, -0.6154,  ...,  0.0365, -0.1763,  0.0686],
         [-0.2336, -0.1466,  0.0708,  ..., -0.0599, -0.1147,  0.0115],
         [ 0.0760, -0.0658, -0.0462,  ..., -0.2356,  0.0167,  0.3105]],

        [[ 0.4001,  0.0139, -0.1037,  ..., -0.0468,  0.6063, -0.2202],
         [-0.0031,  0.0571,  0.0074,  ..., -0.0494,  0.1037, -0.1060],
         [ 0.1458, -0.0303, -0.1922,  ...,  0.2921, -0.1696, -0.0940],
         [-0.0958,  0.3255,  0.3887,  ..., -0.0987, -0.1626, -0.3898]]],
       grad_fn=<UnsafeViewBackward0>)

In [29]:
core.shape

torch.Size([3, 4, 16, 16])

In [45]:
h2 = linearA(x).unsqueeze(-2) @ core

In [43]:
h2.shape

torch.Size([3, 4, 1, 16])

In [32]:
h3 = linearB(h2.squeeze(-2))

In [33]:
h3

tensor([[[ 0.1364,  0.3528, -0.4884,  ...,  0.0451, -0.1590,  0.6033],
         [ 0.9869, -1.3004, -1.6613,  ...,  0.3666, -1.1871,  0.4440],
         [-0.6000,  0.4119,  0.8842,  ...,  0.1745,  0.2319,  0.6379],
         [-0.0575,  0.8677,  2.0496,  ..., -1.1423,  0.7695, -2.2850]],

        [[ 1.2993, -0.9309, -1.9751,  ..., -0.8612, -0.3816, -4.5359],
         [-0.1006,  0.9073, -1.0636,  ...,  0.1978, -0.8493, -0.4151],
         [-0.4874,  0.2984,  0.0530,  ...,  0.2068, -0.2109,  0.3485],
         [ 0.1081, -0.6814,  0.3874,  ..., -0.1913, -0.1430, -0.3207]],

        [[-0.7838,  0.8841, -0.1883,  ...,  0.5851,  1.0157, -0.8078],
         [-0.8792,  1.5166,  0.8756,  ..., -0.7423, -0.0695, -0.1658],
         [-0.1608, -0.1352, -0.1841,  ..., -0.5653, -0.5929, -0.6947],
         [-3.1612,  1.2398,  1.7035,  ..., -1.5426, -0.4911, -1.5156]]],
       grad_fn=<UnsafeViewBackward0>)

In [ ]:
linearA(x).shape

torch.Size([3, 4, 16])

: 

In [ ]:
h2.shape

torch.Size([3, 4, 1, 16])

: 

In [ ]:
linearA(x).unsqueeze(-2).shape

torch.Size([3, 4, 1, 16])

: 

In [2]:
a = torch.load('/data/workspace/projects/moe/AdaMoLE/outputs/qwen3-8b-mole-top2-exp8-math-14k/adapter_model.bin')

In [3]:
a.keys()

dict_keys(['base_model.model.model.layers.0.self_attn.q_proj.lora_A.0.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_A.1.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_A.2.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_A.3.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_A.4.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_A.5.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_A.6.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_A.7.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.0.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.1.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.2.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.3.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.4.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.5.weight', 'base_model.model.model.layers.0.self

In [7]:
b = [x for x in a.keys() if 'moe_layer' in x]

In [9]:
b.__len__()

2880

In [15]:
c = [x for x in a.keys() if 'drop' in x]

In [16]:
c

[]

In [4]:
a['base_model.model.model.layers.0.self_attn.q_proj.moe_layer.experts.0.lora_A.weight']

tensor([[-4.4629e-03, -2.4648e-03, -7.0186e-03,  ...,  1.6958e-02,
         -1.3726e-03,  4.9094e-03],
        [ 4.5847e-03, -1.2500e-02, -1.3889e-02,  ...,  6.0564e-03,
          1.4926e-02, -4.6238e-03],
        [ 9.3714e-03, -1.5020e-02, -1.2793e-02,  ...,  8.6024e-03,
         -3.0532e-03, -8.7763e-03],
        ...,
        [ 9.4404e-03, -3.7128e-04,  6.3849e-03,  ...,  4.1793e-03,
         -1.0146e-02,  6.7490e-04],
        [-3.4635e-03, -6.5786e-03,  1.6937e-02,  ...,  1.2113e-02,
         -6.3088e-03,  1.0748e-02],
        [ 2.2891e-03,  4.8487e-03, -8.5005e-06,  ..., -1.1082e-03,
          8.0367e-03,  1.5279e-03]], device='cuda:0')

In [5]:
a['base_model.model.model.layers.0.self_attn.q_proj.lora_A.0.weight']

tensor([[-4.4629e-03, -2.4648e-03, -7.0186e-03,  ...,  1.6958e-02,
         -1.3726e-03,  4.9094e-03],
        [ 4.5847e-03, -1.2500e-02, -1.3889e-02,  ...,  6.0564e-03,
          1.4926e-02, -4.6238e-03],
        [ 9.3714e-03, -1.5020e-02, -1.2793e-02,  ...,  8.6024e-03,
         -3.0532e-03, -8.7763e-03],
        ...,
        [ 9.4404e-03, -3.7128e-04,  6.3849e-03,  ...,  4.1793e-03,
         -1.0146e-02,  6.7490e-04],
        [-3.4635e-03, -6.5786e-03,  1.6937e-02,  ...,  1.2113e-02,
         -6.3088e-03,  1.0748e-02],
        [ 2.2891e-03,  4.8487e-03, -8.5005e-06,  ..., -1.1082e-03,
          8.0367e-03,  1.5279e-03]], device='cuda:0')

In [10]:
a.keys().__len__()

5940